In [189]:
import pandas as pd
import pdb
from pathlib import Path
from fastai.basics import *
path = Path("UCRArchive_2018")

In [4]:
list((path/"car").iterdir())

[WindowsPath('UCRArchive_2018/car/Car_TEST.tsv'),
 WindowsPath('UCRArchive_2018/car/Car_TRAIN.tsv'),
 WindowsPath('UCRArchive_2018/car/desktop.ini'),
 WindowsPath('UCRArchive_2018/car/README.md')]

In [190]:
class TSCDS(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y
        self.items = [1,2,3]
    
    def __len__(self): return len(self.X)
    
    def __getitem__(self,idx):
        return np.expand_dims(self.X[idx],axis=0), self.Y[idx] - 1

In [206]:
class ConvBlock(torch.nn.Module):
    def __init__(self,inputC,outputC,kernelSize):
        super().__init__()
        self.conv = torch.nn.Conv1d(inputC,outputC,kernelSize)
        self.drop = torch.nn.Dropout()
        self.bn = torch.nn.BatchNorm1d(outputC)
        
    def forward(self,x):
        return self.drop(self.bn(torch.relu(self.conv(x))))

class TSCModel(torch.nn.Module):
    def __init__(self,numClasses):
        super().__init__()
        self.blocks = torch.nn.ModuleList([ConvBlock(i,o,k) for i,o,k in [(1,128,8),(128,256,5),(256,128,3)]])
        self.avgpool = torch.nn.AdaptiveAvgPool1d(1)
        self.out = torch.nn.Linear(128,numClasses)
        
    def forward(self,x):
        x = x.float()
        for b in self.blocks: x = b(x)
        x = self.avgpool(x)
        return self.out(x.squeeze(-1))

In [207]:
def trainNetwork(d,weights=None,epochs=20,lr=5e-2):
    trainDF = pd.read_csv(path/f"{d}/{d}_TRAIN.tsv",sep="\t",header=None)
    testDF = pd.read_csv(path/f"{d}/{d}_TEST.tsv",sep="\t",header=None)
    cat = trainDF.iloc[:,0].astype("category")
    trainDF.iloc[:,0] = cat.cat.codes
    testDF.iloc[:,0] = pd.Categorical(testDF.iloc[:,0],categories=cat.cat.categories).codes
    nClasses = len(cat.cat.categories)
    
    trainDS = TSCDS(trainDF.iloc[:,1:].values,trainDF.iloc[:,0].values)
    testDS = TSCDS(testDF.iloc[:,1:].values,testDF.iloc[:,0].values)
    model = TSCModel(nClasses)
    learn = Learner(data,model,loss_func=torch.nn.functional.cross_entropy,metrics=[accuracy])
    if weights:
        learn.model.blocks.load_state_dict(weights)
    
    learn.fit_one_cycle(epochs,lr)
    return learn.model.blocks.state_dict()

In [193]:
weights = None
for p in path.iterdir():
    if p.is_dir():
        weights = trainNetwork(p.name,weights)

epoch,train_loss,valid_loss,accuracy
1,1.749991,1.465224,0.233333
2,1.574655,1.504321,0.216667
3,1.503467,2.492872,0.233333
4,1.515240,1.740129,0.233333
5,1.459075,1.433599,0.333333
6,1.467439,1.397818,0.350000
7,1.421116,2.365916,0.233333
8,1.423876,1.537795,0.283333
9,1.423254,1.377244,0.316667


KeyboardInterrupt: 

In [208]:
w = trainNetwork("Worms",lr=1e-2,epochs=100)

epoch,train_loss,valid_loss,accuracy
1,1.615551,1.568383,0.416667
2,1.589636,1.557210,0.333333
3,1.563373,1.455474,0.416667
4,1.526425,1.336305,0.500000
5,1.481152,1.226602,0.633333
6,1.417295,1.329268,0.566667
7,1.360600,1.359261,0.300000
8,1.303365,1.243345,0.516667
9,1.257618,1.069968,0.616667
10,1.208771,1.193387,0.483333


In [209]:
trainNetwork("Wine",w,lr=1e-2,epochs=100)

epoch,train_loss,valid_loss,accuracy
1,-4.347342,0.445190,0.450000
2,-3.736607,0.411819,0.466667
3,-3.558654,0.379419,0.466667
4,-3.483701,0.351606,0.466667
5,-3.455778,0.337507,0.466667
6,-1.643021,0.338031,0.466667
7,-0.661304,0.337008,0.466667
8,-0.193588,0.339194,0.466667
9,0.099940,0.340972,0.466667
10,0.171600,0.333909,0.466667


OrderedDict([('0.conv.weight',
              tensor([[[ 0.1073, -0.0872, -0.0879,  ..., -0.1043, -0.0412, -0.0535]],
              
                      [[-0.2463, -0.0052,  0.2100,  ...,  0.0963, -0.1249, -0.1503]],
              
                      [[ 0.2216,  0.1549,  0.1031,  ..., -0.0185,  0.0382, -0.1180]],
              
                      ...,
              
                      [[ 0.5980,  0.6302,  0.6335,  ...,  0.1199,  0.0938,  0.0895]],
              
                      [[-0.0889,  0.2088, -0.0529,  ..., -0.1337, -0.1374,  0.0765]],
              
                      [[ 0.5792,  0.6602,  0.2864,  ...,  0.0325, -0.3880, -0.7142]]],
                     device='cuda:0')),
             ('0.conv.bias',
              tensor([-0.3000, -0.1017, -0.3986, -0.0902, -0.3298, -0.1704,  0.4436, -0.2207,
                       0.2127,  0.3475,  0.1311, -0.2273,  0.4033, -0.0943,  0.0187, -0.1235,
                      -0.2214, -0.1711, -0.5492, -0.6197,  0.8057, -0.4565, -0